In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install gensim==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 67.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for gensim
  Running setup.py clean for gensim
Failed to build gensim
ERROR: Could not build wheels for gensim, which is required to install pyproject.toml-based projects


In [4]:
from collections import defaultdict, Counter
from typing import List, Dict, Union
import pickle

from tqdm import tqdm
import numpy as np
import polars as pl
from gensim.models import Word2Vec

In [5]:
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
TOP_N = 50
LOCALES = ["IT", "FR", "ES"]
VER = "17"
SEED = 42

In [6]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [7]:
def train_word2vec(df:pl.DataFrame) -> Word2Vec:

    # sessionごとにaidのシーケンスを作成
    aid_sequences = list(df.groupby("session_id", maintain_order=True).all()["prev_items"].to_list())

    # word2vecの学習
    model = Word2Vec(
        sentences=aid_sequences,
        epochs=50,
        workers=8,
        min_count=1,
        seed=SEED,
    )

    return model

In [8]:
def make_nns_matrix(w2v, k):
    aid_xs = []
    aid_ys = []
    sims = []
    for aid in tqdm(w2v.wv.index_to_key):
        nns = w2v.wv.most_similar(aid, topn=k)
        aid_y = [x[0] for x in nns]
        sim = [x[1] for x in nns]
        aid_xs.extend([aid] * k)
        aid_ys.extend(aid_y)
        sims.extend(sim)

    return pl.DataFrame({"item": aid_xs, "candidate_item": aid_ys, 'nns_similality': sims})

# For local train/eval

In [9]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2_augmented.parquet")
test = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_leftover.parquet")

In [10]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])

In [11]:
# train word2vec model
for locale in LOCALES:
    # filter by locale
    df = session_df.filter(pl.col("locale") == locale)

    # train and save word2vec model
    model = train_word2vec(df)
    model.save(DIR + f"models/task2/item2vec_{locale}_{VER}_for_train_or_eval.model")

## MRR@100

In [12]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task2/train_task2.parquet")

In [13]:
# last_itemの抽出
last_item_list = []
prev_items_list = train["prev_items"].to_list()
for prev_items in prev_items_list:
    last_item_list.append(prev_items[-1])
train = train.with_columns(pl.Series(name="last_item", values=last_item_list))

In [14]:
train = train[["session_id", "locale", "last_item", "next_item"]]

In [15]:
nns_matrices = []
for locale in LOCALES:
    # calculate nearest neighbors
    w2v = Word2Vec.load(DIR + f"models/task2/item2vec_{locale}_{VER}_for_train_or_eval.model")
    nns_matrix = make_nns_matrix(w2v, TOP_N)
    nns_matrix = nns_matrix.with_columns(pl.lit(locale).alias("locale"))
    nns_matrices.append(nns_matrix)
nns_matrix = pl.concat(nns_matrices)

100%|██████████| 40579/40579 [01:05<00:00, 619.94it/s]


In [16]:
file_name = f"nns_matrix_{VER}_for_train_or_eval.parquet"
nns_matrix.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task2/" + file_name)

In [17]:
train = train.join(nns_matrix, left_on=["locale", "last_item"], right_on=["locale", "item"], how="left")
train = train.sort(["session_id", "nns_similality"], descending=[False, True])
train = train.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
label_lists = train.groupby("session_id", maintain_order=True).all()["label"].to_list()

In [18]:
# MRRの計算
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.23315


# For test inference

In [19]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2_augmented.parquet")
test = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_leftover.parquet")

In [20]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [21]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])

In [22]:
# train word2vec model
for locale in LOCALES:
    # filter by locale
    df = session_df.filter(pl.col("locale") == locale)

    # train and save word2vec model
    model = train_word2vec(df)
    model.save(DIR + f"models/task2/item2vec_{locale}_{VER}_for_inference.model")

In [23]:
nns_matrices = []
for locale in LOCALES:
    # calculate nearest neighbors
    w2v = Word2Vec.load(DIR + f"models/task2/item2vec_{locale}_{VER}_for_inference.model")
    nns_matrix = make_nns_matrix(w2v, TOP_N)
    nns_matrix = nns_matrix.with_columns(pl.lit(locale).alias("locale"))
    nns_matrices.append(nns_matrix)
nns_matrix = pl.concat(nns_matrices)

100%|██████████| 42503/42503 [01:07<00:00, 628.29it/s]


In [24]:
file_name = f"nns_matrix_{VER}_for_inference.parquet"
nns_matrix.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task2/" + file_name)